In [1]:
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures


C:\Users\Arif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh_trimesh = trimesh.load_mesh("duzen2.stl")

# Get vertices, faces, and normals
vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)


# Eğrilik ölçümü (örnek olarak kullanılıyor)
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, radius=2)


In [3]:
def get_largest_cavity(vertices, faces, mean_curvature):
    # Calculate curvature threshold to define cavities
    curvature_threshold = np.percentile(mean_curvature, 14)
    
    # Find indices of vertices in cavities
    cavity_indices = np.where(mean_curvature < curvature_threshold)[0]
    
    # Get faces corresponding to cavity vertices
    cavity_faces = faces[np.isin(faces, cavity_indices).any(axis=1)]
    
    # Find unique vertices in cavity faces
    cavity_vertices = np.unique(cavity_faces)
    
    # If no cavities found, return empty array
    if len(cavity_vertices) == 0:
        return np.array([])
    
    # Cluster cavity vertices to distinguish separate cavities
    cavity_points = vertices[cavity_vertices]
    
    # Use DBSCAN to cluster cavity regions
    clustering = DBSCAN(eps=1.0, min_samples=5).fit(cavity_points)
    labels = clustering.labels_
    
    # Find the largest cavity cluster
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    
    # Exclude noise points (label -1)
    valid_labels = unique_labels[unique_labels != -1]
    valid_counts = label_counts[unique_labels != -1]
    
    if len(valid_labels) == 0:
        return np.array([])
    
    # Get the label of the largest cavity cluster
    largest_cavity_label = valid_labels[np.argmax(valid_counts)]
    
    # Get indices of vertices in the largest cavity
    largest_cavity_mask = labels == largest_cavity_label
    largest_cavity_vertices = cavity_vertices[largest_cavity_mask]
    
    return largest_cavity_vertices

In [4]:
def extract_cavity_bottom_surface(vertices, faces, cavity_vertices):
    """
    Extract the bottom surface indices of the cavity using RANSAC.
    
    Args:
        vertices (np.ndarray): All mesh vertices
        faces (np.ndarray): All mesh faces
        cavity_vertices (np.ndarray): Indices of vertices in the cavity
    
    Returns:
        np.ndarray: Indices of vertices forming the bottom surface of the cavity
    """
    # Get the cavity vertices coordinates
    cavity_points = vertices[cavity_vertices]
    # Prepare data for RANSAC
    # Use x and y coordinates to predict z
    X = cavity_points[:, :2]  # x and y coordinates
    y = cavity_points[:, 2]   # z coordinate
    
    # Use polynomial features to fit a plane
    poly_features = PolynomialFeatures(degree=1, include_bias=False)
    X_poly = poly_features.fit_transform(X)
    
    # RANSAC regression to find the bottom surface plane
    ransac = RANSACRegressor(
        random_state=42, 
        max_trials=100,
        residual_threshold=1.0
    )
    
    # Fit RANSAC
    ransac.fit(X_poly, y)
    
    # Predict z values for all cavity points
    y_pred = ransac.predict(poly_features.transform(X))
    
    # Find points close to the estimated plane
    residuals = np.abs(y - y_pred)
    
    # Define bottom surface as points within a small threshold of the plane
    bottom_surface_mask = residuals <= np.percentile(residuals, 65)
    
    # Get indices of bottom surface vertices
    bottom_surface_indices = cavity_vertices[bottom_surface_mask]
    
    return bottom_surface_indices

In [ ]:

cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
outline_indices = np.where((mean_curvature > 2.5))[0]
cavity_faces = faces[cavity_indices]
curvature_threshold = np.percentile(mean_curvature, 14)


cavity_vertices = vertices[np.unique(cavity_faces)]

largest_cavity = get_largest_cavity(vertices, faces, mean_curvature)

cavity_surface_vertices= extract_cavity_bottom_surface(vertices,faces,largest_cavity)


mesh_o3d = o3d.geometry.TriangleMesh()
mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
mesh_o3d.triangles = o3d.utility.Vector3iVector(faces)
mesh_o3d.compute_vertex_normals()

colors = np.ones((vertices.shape[0], 3)) * 0.7  # Light gray for normal surface
colors[outline_indices] = [1, 0, 0]  # Red for outline
colors[cavity_surface_vertices] = [0, 1, 0]  # Green for cavities

mesh_o3d.vertex_colors = o3d.utility.Vector3dVector(colors)
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3)
# Visualize tooth with cavity detection
o3d.visualization.draw_geometries([mesh_o3d,coordinate_frame])